In [1]:
import numpy as np 
import pandas as pd
from tqdm import tqdm

import sys
sys.path.append('../')
from retinanet import coco_eval
from retinanet import csv_eval
from retinanet import model
from retinanet import paps_eval
from retinanet import paps_train

# from retinanet import retina
from retinanet.dataloader import *
from retinanet.anchors import Anchors
from retinanet.losses import *
from retinanet.scheduler import *
from retinanet.hourglass import hg1, hg2, hg8
from retinanet.parallel import DataParallelModel, DataParallelCriterion

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler
import torch.optim as optim


In [2]:
# device = torch.device('cpu')
device = torch.device('cuda')
model = hg2(device, pretrained=True, progress=False, num_classes=2)
model.to(device)

num_classes 2


HourglassNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
  )
  (layer2): Sequential(
    (0): Bottleneck(
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=Tr

In [3]:
# model

In [4]:

# train_info = np.load('../data/train.npy', allow_pickle=True, encoding='latin1').item()
# train_info

batch_size = 24
dataset_train = PapsDataset('../data/', set_name='train_2class',
                            transform=train_transforms)

train_data_loader = DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True,
    num_workers=12,
    pin_memory=True,
    prefetch_factor=1,
    collate_fn=collate_fn
)

loading annotations into memory...
Done (t=0.27s)
creating index...
index created!


In [5]:
criterion = PapsLoss(device)
criterion = criterion.to(device)
model.training = True

# https://gaussian37.github.io/dl-pytorch-lr_scheduler/
optimizer = optim.Adam(model.parameters(), lr = 1e-8)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=20, T_mult=2, eta_max=0.0008,  T_up=5, gamma=0.5)
# CosineAnnealingWarmRestarts

In [6]:
saved_dir = '../trained_models/HourGlass/loss_filter/'
s_epoch= 0
e_epoch= 120

In [7]:
if os.path.isdir(saved_dir) == False :
    print('folder is made')
    os.makedirs(saved_dir)

In [8]:
if os.path.isfile(saved_dir + 'epoch_' + str(e_epoch) +'_model.pt') :
    print('pretrainind file loading')
    state = torch.load(saved_dir + 'epoch_' + str(e_epoch) +'_model.pt')
    epoch = state['epoch']
    model.load_state_dict(state['state_dict'], strict=False)
    optimizer.load_state_dict(state['optimizer'])
    last_loss = state['loss']
else :
    last_loss = 0.6

In [ ]:
paps_train.train_paps(dataloader=train_data_loader, 
                      model=model, 
                      criterion=criterion,
                      saved_dir=saved_dir, 
                      optimizer=optimizer,
                      scheduler=scheduler,
                      device = device,
                      s_epoch= s_epoch,
                      e_epoch= e_epoch,
                      last_loss = last_loss)  

  0%|          | 0/620 [00:00<?, ?it/s]                                                                                                                          

0th epochs loss 1.1326649719668973 lr 1e-08 


  0%|          | 0/620 [00:00<?, ?it/s]                                                                                                                               

1th epochs loss 1.2364010437845419 lr 0.000160008 


  0%|          | 0/620 [00:00<?, ?it/s]                                                                                                                                

...

  0%|          | 0/620 [00:00<?, ?it/s]                                                                                                                                    

74th epochs loss 0.09235944930249108 lr 0.00017431576682361204 
best model is saved


  0%|          | 0/620 [00:00<?, ?it/s]                                                                                                                                    

75th epochs loss 0.09864165507352506 lr 0.00017071214258474883 


 22%|██▏       | 138/620 [01:27<05:39,  1.42it/s, avg_loss=0.0948, batch_loss=0.128, cls_loss=0.125, lr=0.000167, num_det=tensor(42, device='cuda:0'), reg_loss=0.00329]   

In [ ]:
# state = {
#     'epoch': 0,
#     'state_dict': model.state_dict(),
#     'optimizer': optimizer.state_dict(),
#     'loss' : 0.6
# }
# torch.save(state, saved_dir + 'model.pt')

In [ ]:
dataset_val = PapsDataset('../data/', set_name='val_2class',
                            transform=val_transforms)

val_data_loader = DataLoader(
    dataset_val,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
paps_eval.evaluate_paps(dataset=dataset_val, 
  dataloader=val_data_loader, 
  model=model, 
  saved_dir=saved_dir, 
  device = device,
  threshold=0.5) 